In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2018-1-2 --end 2018-10-25 -b futures
import pandas as pd
import numpy as np
import talib
from zipline.api import order, record, symbol, continuous_future, future_symbol
from strategies.contracts import contracts


FAST_MA = 50
SLOW_MA = 100
BREAKOUT = 20
STOP = 2

def initialize(context):
    context.cont_contracts = [
        continuous_future(contract,
                          adjustment='mul',
                          roll='volume')
        for contract in contracts]

        

def handle_data(context, data):
    context.df = data.history(context.cont_contracts,
                      'price',
                      bar_count = SLOW_MA + 10,
                      frequency = '1d')

def analyze(context, data):
    print(context.df.head())


Exception ignored in: 'zipline.assets.continuous_futures.OrderedContracts.contract_before_auto_close'
AttributeError: 'NoneType' object has no attribute 'next'


KeyError: 0